# 4.3 지도시각화

###  4.3.2 데이터 준비

In [ ]:
# 예제 4-20 크롤링 데이터 불러오기
import pandas as pd
raw_total = pd.read_excel('./files/3_1_crawling_raw.xlsx')
raw_total.head()

In [ ]:
# 예제 4-21 위치정보 가져오기
location_counts = raw_total['place'].value_counts( )
location_counts

In [ ]:
# 예제 4-22 등록된 위치정보별 빈도수 데이터
location_counts_df = pd.DataFrame(location_counts)
location_counts_df.head()

In [ ]:
# 예제 4-23 위치정보 빈도수 데이터 저장하기
location_counts_df.to_excel('./files/3_3_location_counts.xlsx')

In [ ]:
# 예제 4-24 위치정보 종류 확인하기
locations = list( location_counts.index )
locations 

### 4.3.4 네이버 지도 API를 활용한 장소 검색

In [ ]:
# 예제 4-25 네이버지도 API 사용 예시
import requests

searching = '합정 스타벅스'
base_url = 'https://naveropenapi.apigw.ntruss.com/map-place/v1/search'
query_string = '?query={}&coordinate=126.977967,37.566329'.format(searching)
url = base_url + query_string

client_id = 'y86xoxxxxx'         
client_secret = 'cBxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

headers = {'X-NCP-APIGW-API-KEY-ID': client_id, 
           'X-NCP-APIGW-API-KEY': client_secret} 

places = requests.get(url, headers = headers).json()['places']
places


In [ ]:
# 예제 4-26 Search Place API 장소 검색 함수 만들기
def find_places(searching):
    # ① 접속URL 만들기
    base_url = 'https://naveropenapi.apigw.ntruss.com/map-place/v1/search'
    query_string = '?query={}&coordinate=126.977967,37.566329'.format(searching)
    url = base_url + query_string
    # ② headers 입력하기
    client_id = 'y86xoxxxxx'         
    client_secret = 'cB2gj68owMnDi45GgcoA2SomRV6gCxxxxxxxxxxx'
    headers = {'X-NCP-APIGW-API-KEY-ID': client_id, 
                   'X-NCP-APIGW-API-KEY': client_secret} 
    # ③ API 요청&정보 받기
    places = requests.get(url, headers = headers).json()['places']  
    # ④ 필요한 정보 선택하기
    place = places[0] 
    name = place['name']
    x=place['x']
    y=place['y']
    data = [name, x, y, searching] 

    return data


In [ ]:
# 예제 4-27 제주공항 검색 예시
data = find_places('제주공항')
data

In [ ]:
# 예제 4-28 인스타그램 위치명 위치정보 검색하기
import time

locations_inform = [ ]
for location in locations:
    try:
        data = find_places(location)       
        locations_inform.append(data) 
        time.sleep(0.5) 
    except:
        pass
locations_inform


In [ ]:
# 예제 4-29 위치정보 저장하기
locations_inform_df = pd.DataFrame(locations_inform)
locations_inform_df.columns = ['네이버위치명','경도','위도','인스타위치명']
locations_inform_df.to_excel('./files/3_3_locations.xlsx', index=False)


In [ ]:
# 예제 4-30 인스타 게시량 및 위치정보 데이터 불러오기
location_counts_df = pd.read_excel('./files/3_3_location_counts.xlsx')
locations_inform_df = pd.read_excel('./files/3_3_locations.xlsx')

In [ ]:
# 예제 4-31 위치 데이터 병합하기
location_data = pd.merge(locations_inform_df, location_counts_df, 
                         how = 'inner', left_on = 'name_official', right_index=True)
location_data.head()


In [ ]:
# 예제 4-32 병합한 데이터 저장하기
location_data.to_excel('./files/3_3_location_inform.xlsx', index=False)

### 4.3.6 folium을 이용한 지도 시각화 ① - 개별 표시

In [ ]:
# 예제 4-33 데이터 불러오기
location_data = pd.read_excel('./files/3_3_location_inform.xlsx')
location_data.info()

In [ ]:
# 예제 4-34 지도 표시하기
import folium

Mt_Hanla =[33.362500, 126.533694]
map_jeju = folium.Map(location = Mt_Hanla, zoom_start = 11)

for i in range(len(location_data)):
    name = location_data ['name_official'][i]    # 공식명칭
    count = location_data ['place'][i]           # 게시글 개수
    size = int(count)*2
    long = float(location_data['y'][i])      
    lat = float(location_data['x'][i])       
    folium.CircleMarker((long,lat), radius = size, color='red', popup=name).add_to(map_jeju)
    
map_jeju

In [ ]:
# 예제 4-35 지도 저장하기
map_jeju.save('./files/3_3_jeju.html') 

### 4.3.7 folium을 이용한 지도 시각화 ② - 그룹으로 표시

In [ ]:
# 예제 4-36 지도 표시하기(마커 집합)
from folium.plugins import MarkerCluster

locations = []
names = []

for i in range(len(location_data)):
    data = location_data.iloc[i]  # 행 하나씩
    locations.append((float(data['y']),float(data['x'])))    # 위도 , 경도 순으로..
    names.append(data['name'])

icon_create_function = """\
function(cluster) {
    return L.divIcon({
    html: '<b>' + cluster.getChildCount() + '</b>',
    className: 'marker-cluster marker-cluster-large',
    iconSize: new L.Point(30, 30)
    });
}"""

Mt_Hanla =[33.362500, 126.533694]
map_jeju2 = folium.Map(location = Mt_Hanla, zoom_start = 11)
                       
marker_cluster = MarkerCluster(
    locations=locations, popups=names,
    name='Jeju',
    overlay=True,
    control=True,
    icon_create_function=icon_create_function
)

marker_cluster.add_to(map_jeju2)
folium.LayerControl().add_to(map_jeju2)

map_jeju2


In [ ]:
# 예제 4-37 지도 저장하기
map_jeju2.save('./files/3_3_jeju_cluster.html') 